In [ ]:
# install deepxde

%pip install deepxde

In [ ]:
# import packages

# import deepxde as dde
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os

if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

torch.cuda.is_available()

device = torch.device("cuda")

In [ ]:
# define the PDE

def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t + y * dy_x - 0.01 / np.pi * dy_xx

# define a computational geometry and a time domain

geom = dde.geometry.Interval(-1, 1)
timedomain = dde.geometry.TimeDomain(0, 0.5)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# define the boundary and initial conditions

bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(geomtime, lambda x: -np.sin(np.pi * x[:, 0:1]), lambda _, on_initial: on_initial)

# define the time PDE problem

data = dde.data.TimePDE(geomtime, pde, [bc, ic],
                        num_domain=25400, num_boundary=80, num_initial=160)

In [ ]:
# choose the network architecture and the training method

net = dde.nn.FNN([2] + [20] * 3 + [1], "tanh", "Glorot normal")

model = dde.Model(data, net)

model.compile("adam", lr=1e-3, loss_weights=[1, 1, 1])

In [ ]:
# train the model

losshistory, train_state = model.train(iterations=10000)

In [ ]:
# predict the solution at t = 0.5 for x in [-1, 1]

x = geom.uniform_points(256, True)
t = np.full(256, 0.5)
x = np.hstack((x, t.reshape(-1, 1)))
y = model.predict(x)

In [ ]:
# plot the solution and compare it to the sine function

plt.plot(x[:, 0], y[:, 0], label="PINN")
plt.plot(x[:, 0], -np.sin(np.pi * x[:, 0]), label="sine")
plt.legend()
plt.title("Functions learned at t = 0")
plt.xlabel("x")
plt.ylabel("u(x, t)")
plt.show()

In [12]:
# import data for the true solution

def gen_testdata():
    data = np.load("c:/Users/Lukas/OneDrive/Dokumente/GitHub/CS229-Foundations-of-Deep-Learning/DPM Results Reproduction/Burgers.npz")
    t, x, exact = data["t"], data["x"], data["usol"].T
    xx, tt = np.meshgrid(x, t)
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    y = exact.flatten()[:, None]
    return X, y

X, y_true = gen_testdata()

: 

: 